In [3]:
import torch
from segment_anything import SamPredictor, sam_model_registry

In [6]:
sam_model = sam_model_registry['vit_b'](checkpoint='C:/SAM/segment-anything/checkpoints/sam_vit_b_01ec64.pth')
optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters()) 
loss_fn = torch.nn.MSELoss()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/SAM/segment-anything/checkpoints/sam_vit_b_01ec64.pth'

In [ ]:
def training(config, model, train_loader, valid_loader):
    model = model.to(config['device'])
    es_count = 0
    min_val_loss = float('inf')
    best_model = None
    
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['train']['lr'])

    # training loop
    for epoch in range(config['train']['epochs']):
        model.train()
        epoch_loss = 0
        for images, masks in tqdm(train_loader):
            images = images.float().to(config['device'])
            masks = masks.float().to(config['device'])

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks.unsqueeze(1))
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        val_loss = validation(config, model, criterion, valid_loader)
        es_count += 1
        if min_val_loss > val_loss:
            es_count = 0
            min_val_loss = val_loss
            best_model = model
            best_epoch = epoch
            print(f"Epoch [{epoch + 1}] New Minimum Valid Loss!")
        
        if es_count == config['early_stopping']:
            print(f"EARLY STOPPING COUNT: {config['early_stopping']} BEST EPOCH: {best_epoch}")
            return best_model
        
        print(f'Epoch {epoch+1}, Train Loss: {epoch_loss/len(train_loader)}', 'Valid Loss:', val_loss, 'ES Count:', es_count)
        print("------------------------------------------------------------------------------------")
    
    print(f"EARLY STOPPING COUNT에 도달하지 않았습니다! \nEARLY STOPPING COUNT: {config['early_stopping']} BEST EPOCH: {best_epoch}")
    return best_model